<a href="https://colab.research.google.com/github/ekyuho/2024-1-iot/blob/main/whatsmatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json

api_key=''
url='http://175.197.63.177:8000/local/api/'

def run(t, method, h, j, p):
    headers={'x-api-key':api_key}
    headers.update(h)
    print(url+t, method, p, j)
    if method=='get': r=requests.get(url+t, headers=headers, params=p, timeout=10)
    elif method=='post': r=requests.get(url+t, headers=headers, json=j, timeout=10)
    elif method=='put': r=requests.put(url+t, headers=headers, json=j, timeout=10)
    return r

In [4]:
print('1. 사용가능한 모든 디바이스 찾기')
r=run('devices', 'get', h={}, j={}, p={})
d1=r.json()
print(f'got {len(d1)} devices','\n')

# 한집의 표준 모델, 즉 어떤 대상 집에는 다음 4가지의 디바이스가 있다고 가정, Query를 통해 유무 확인
#  entity_id: 표준 명칭
#  location: 특정 장치의 ID
# 10월10일 스마트라이프 전시에는 다음 네가지 디바이스를 설치함.
house={'main-light':{'entity_id':'light.nano_vdevo', 'location':''},
       'desk-light':{'entity_id':'switch.otopeulreiseu_kaendeulweomeo', 'location':''},
       'curtain':{'entity_id':'', 'location':''},
       'presense-sensor':{'entity_id':'binary_sensor.letianpai_presence_sensor_box_occupancy_2', 'location':''}
      }
rooms=set([])

def match(d):
    global house, rooms
    for x in house:
        if house[x]['entity_id']==d['entity_id']:
            house[x]['location']=d['location']
            house[x]['alias']=d['alias']
            rooms.add(d['location'])

            #r=run(f"devices/{d['entity_id']}/services", 'get', h={}, j={}, p={})  # 실제 사용하는 케이스 없음.
            #house[x]['service']=r.json()
            return True
    return False

for d in d1:
    if match(d):
        print('matched', d)

print('rooms=', rooms)

1. 사용가능한 모든 디바이스 찾기
http://175.197.63.177:8000/local/api/devices get {} {}
got 10 devices 

matched {'alias': '라이트 나노 램프', 'entity_id': 'light.nano_vdevo', 'location': 'test_room_id_1724216213468'}
matched {'alias': '캔들워머램프 김동은 프로 사무실', 'entity_id': 'switch.otopeulreiseu_kaendeulweomeo', 'location': 'test_room_id_1724226467853'}
matched {'alias': 'LetianPai 재실센서', 'entity_id': 'binary_sensor.letianpai_presence_sensor_box_occupancy_2', 'location': 'test_room_id_1724320457362'}
rooms= {'test_room_id_1724226467853', 'test_room_id_1724216213468', 'test_room_id_1724320457362'}


In [5]:
print('2. 각 디바이스 상태')
for x in house:
    d=house[x]
    r=run(f"states/{d['entity_id']}", 'get', h={}, j={}, p={})
    try:
        d['state']=r.json()['state']
    except:
        d['state']=''
    print(d)

2. 각 디바이스 상태
http://175.197.63.177:8000/local/api/states/light.nano_vdevo get {} {}
{'entity_id': 'light.nano_vdevo', 'location': 'test_room_id_1724216213468', 'alias': '라이트 나노 램프', 'state': 'on'}
http://175.197.63.177:8000/local/api/states/switch.otopeulreiseu_kaendeulweomeo get {} {}
{'entity_id': 'switch.otopeulreiseu_kaendeulweomeo', 'location': 'test_room_id_1724226467853', 'alias': '캔들워머램프 김동은 프로 사무실', 'state': 'off'}
http://175.197.63.177:8000/local/api/states/ get {} {}
{'entity_id': '', 'location': '', 'state': ''}
http://175.197.63.177:8000/local/api/states/binary_sensor.letianpai_presence_sensor_box_occupancy_2 get {} {}
{'entity_id': 'binary_sensor.letianpai_presence_sensor_box_occupancy_2', 'location': 'test_room_id_1724320457362', 'alias': 'LetianPai 재실센서', 'state': 'off'}


In [6]:
print('3. 전등/스위치 on/off')
for x in house:
    d=house[x]
    data={
        "domain": "light",
        "service" : "turn_on",
        "rgb_color" : [255,255,255]
    }
    r=run(f"devices/{d['entity_id']}/command", 'post', h={}, j=data, p={})  # 일단 light와 switch에 대해 tunn_on 하면 허용안된다는 405 코드리턴.
    print(r)

3. 전등/스위치 on/off
http://175.197.63.177:8000/local/api/devices/light.nano_vdevo/command post {} {'domain': 'light', 'service': 'turn_on', 'rgb_color': [255, 255, 255]}
<Response [405]>
http://175.197.63.177:8000/local/api/devices/switch.otopeulreiseu_kaendeulweomeo/command post {} {'domain': 'light', 'service': 'turn_on', 'rgb_color': [255, 255, 255]}
<Response [405]>
http://175.197.63.177:8000/local/api/devices//command post {} {'domain': 'light', 'service': 'turn_on', 'rgb_color': [255, 255, 255]}
<Response [404]>
http://175.197.63.177:8000/local/api/devices/binary_sensor.letianpai_presence_sensor_box_occupancy_2/command post {} {'domain': 'light', 'service': 'turn_on', 'rgb_color': [255, 255, 255]}
<Response [405]>
